# Weak References - Part 2 of 2 (the weird stuff)

## Review: Other proxy object in the standard library

We've covered two other kinds of proxy objects:

### Mapping proxies

The `__dict__` descriptor on a class returns a mapping proxy, which prohibits mutating operations such as setting a different value for a key:

In [1]:
list.__dict__

mappingproxy({'__new__': <function list.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'list' objects>,
              '__hash__': None,
              '__getattribute__': <slot wrapper '__getattribute__' of 'list' objects>,
              '__lt__': <slot wrapper '__lt__' of 'list' objects>,
              '__le__': <slot wrapper '__le__' of 'list' objects>,
              '__eq__': <slot wrapper '__eq__' of 'list' objects>,
              '__ne__': <slot wrapper '__ne__' of 'list' objects>,
              '__gt__': <slot wrapper '__gt__' of 'list' objects>,
              '__ge__': <slot wrapper '__ge__' of 'list' objects>,
              '__iter__': <slot wrapper '__iter__' of 'list' objects>,
              '__init__': <slot wrapper '__init__' of 'list' objects>,
              '__len__': <slot wrapper '__len__' of 'list' objects>,
              '__getitem__': <method '__getitem__' of 'list' objects>,
              '__setitem__': <slot wrapper '__setitem__' of

We can also construct mapping proxies explicitly using the convenient `MappingProxyType` alias in the `types` module.

In [2]:
import types

In [3]:
dir(types)

['AsyncGeneratorType',
 'BuiltinFunctionType',
 'BuiltinMethodType',
 'CellType',
 'ClassMethodDescriptorType',
 'CodeType',
 'CoroutineType',
 'DynamicClassAttribute',
 'EllipsisType',
 'FrameType',
 'FunctionType',
 'GeneratorType',
 'GenericAlias',
 'GetSetDescriptorType',
 'LambdaType',
 'MappingProxyType',
 'MemberDescriptorType',
 'MethodDescriptorType',
 'MethodType',
 'MethodWrapperType',
 'ModuleType',
 'NoneType',
 'NotImplementedType',
 'SimpleNamespace',
 'TracebackType',
 'UnionType',
 'WrapperDescriptorType',
 '_GeneratorWrapper',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_calculate_meta',
 '_cell_factory',
 'coroutine',
 'new_class',
 'prepare_class',
 'resolve_bases']

In [4]:
help(types.MappingProxyType)

Help on class mappingproxy in module builtins:

class mappingproxy(object)
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __or__(self, value, /)
 |      Return self|value.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __reversed__(...)
 |      D.__reversed__() -> r

In [5]:
d = {1: 1, 2: 2, 3: 3}

In [6]:
md = types.MappingProxyType(d)

In [7]:
md 

mappingproxy({1: 1, 2: 2, 3: 3})

In [8]:
md[2]

2

In [9]:
md[2] = 3

TypeError: 'mappingproxy' object does not support item assignment

In [10]:
d[2] = 4

In [11]:
md[2]  # It's a proxy, so it reflects changes to the object "behind" it.

4

### `super` proxies

In [12]:
class Base: 
    def f(self):
        print("Base.f")

In [13]:
class Derived(Base): 
    def f(self): 
        print("Derived.f")
        super().f()

In [14]:
Derived().f()

Derived.f
Base.f


## Weak reference proxies

In [15]:
import weakref

In [16]:
class C: 
    def __init__(self): 
        self.a = 'a'
        self.b = 'b'

In [17]:
c = C()

In [18]:
wcsimple = weakref.proxy(c)

In [19]:
c.a

'a'

In [20]:
wcsimple.a

'a'

In [21]:
del c

In [22]:
wcsimple.a

ReferenceError: weakly-referenced object no longer exists

In [23]:
c = C()

In [24]:
c.a

'a'

In [25]:
wc = weakref.proxy(c, lambda x: print(f"{x}'s referant has died."))

In [26]:
wc

In [27]:
wc.a

'a'

In [28]:
del c

Exception ignored in: <function <lambda> at 0x0000025B3246A7A0>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_17504\3266874697.py", line 1, in <lambda>
ReferenceError: weakly-referenced object no longer exists


In [30]:
wc

ReferenceError: weakly-referenced object no longer exists

In [31]:
c = C()

In [32]:
cghost = weakref.proxy(c, lambda x: print("Boooooooooooooooo!"))

In [33]:
del c

Boooooooooooooooo!


In [34]:
c = C()

In [35]:
rptest = weakref.proxy(c)

In [36]:
repr(c)

'<__main__.C object at 0x0000025B3249D6D0>'

In [37]:
repr(rptest)

'<weakproxy at 0x0000025B326CB470 to C at 0x0000025B3249D6D0>'

In [38]:
rptest

In [39]:
rptest.__repr__()

'<__main__.C object at 0x0000025B3249D6D0>'

In [40]:
class DiffBox: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'

    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        print(f'DiffBox.__eq__({self!r}, {other!r})')
        return self.value == other.value
    
    def __sub__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        print(f'DiffBox.__sub__({self!r}, {other!r})')
        return type(self)(self.value - other.value)

In [41]:
d1 = DiffBox(1)

In [42]:
d5 = DiffBox(5)

In [43]:
d4 = d5 - d1

DiffBox.__sub__(DiffBox(5), DiffBox(1))


In [44]:
d4

DiffBox(4)

In [45]:
d5 == DiffBox(5)

DiffBox.__eq__(DiffBox(5), DiffBox(5))


True

In [46]:
d5 == DiffBox(4)

DiffBox.__eq__(DiffBox(5), DiffBox(4))


False

In [47]:
d20 = DiffBox(20)

In [48]:
d10 = DiffBox(10)

In [49]:
wd20 = weakref.proxy(d20)

In [50]:
wd10 = weakref.proxy(d10)

In [51]:
wd20 == wd10

DiffBox.__eq__(DiffBox(20), DiffBox(10))


False

In [52]:
wd20 - wd10

DiffBox.__sub__(DiffBox(20), DiffBox(10))


DiffBox(10)

In [53]:
result = wd20 - wd10

DiffBox.__sub__(DiffBox(20), DiffBox(10))


In [54]:
result - wd10

DiffBox.__sub__(DiffBox(10), <weakproxy at 0x0000025B327009F0 to DiffBox at 0x0000025B32679A50>)


DiffBox(0)

In [55]:
type(wd10)

weakref.ProxyType

In [56]:
isinstance(wd10, DiffBox)

True

In [57]:
isinstance(wd10, weakref.ProxyType)

True

In [58]:
issubclass(type(wd10), DiffBox)

False

In [59]:
type(wd10).__mro__

(weakref.ProxyType, object)

In [60]:
type(d20)

__main__.DiffBox

In [61]:
d20.__class__

__main__.DiffBox

In [62]:
wd20.__class__

__main__.DiffBox

In [63]:
C.__dict__  # No __class__. So... we can define it?!

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.C.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

In [64]:
class Liar: 
    __class__ = int

In [65]:
issubclass(Liar, int)

False

In [66]:
isinstance(Liar(), int)

True

In [67]:
C.__class__

type

In [68]:
Liar.__class__

type

In [69]:
c = C()

In [70]:
c.__dict__

{'a': 'a', 'b': 'b'}

In [71]:
c.__dict__['__class__'] = int

In [72]:
c.__dict__

{'a': 'a', 'b': 'b', '__class__': int}

In [73]:
c.__class__

__main__.C

In [74]:
li = Liar()

In [75]:
li.__class__

int

In [76]:
li.__dict__['__class__'] = list 

In [77]:
li.__class__

list

In [78]:
class P:
    @property
    def x(self):
        return 10

In [79]:
p = P()

In [80]:
p.__dict__['x'] = 20

In [81]:
p.x

10

In [82]:
class LiarAlt: 
    @property
    def __class__(self): 
        return int

In [83]:
lia = LiarAlt()

In [84]:
lia.__class__

int

In [85]:
isinstance(lia, int) 

True

In [86]:
issubclass(LiarAlt, int)

False

In [87]:
class Holds: 
    def __init__(self, thing): 
        self._thing = thing
    
    @property
    def __class__(self): 
        return self._thing.__class__

In [88]:
hi = Holds(1)

In [89]:
hi._thing

1

In [90]:
isinstance(hi, int)

True

In [91]:
hh = Holds(hi)

In [92]:
isinstance(hh, int)

True

In [93]:
class NarrowDiffBox: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'

    def __eq__(self, other): 
        if type(self) is not type(other): 
            return NotImplemented
        print(f'NarrowDiffBox.__eq__({self!r}, {other!r})')
        return self.value == other.value
    
    def __sub__(self, other): 
        if type(self) is not type(other):
            print(f'Typecheck denied in NarrowDiffBox.__sub__({self!r}, {other!r})')
            return NotImplemented
        print(f'NarrowDiffBox.__sub__({self!r}, {other!r})')
        return type(self)(self.value - other.value)

In [94]:
d1 = NarrowDiffBox(1)

In [95]:
d5 = NarrowDiffBox(5)

In [96]:
d4 = d5 - d1

NarrowDiffBox.__sub__(NarrowDiffBox(5), NarrowDiffBox(1))


In [97]:
d4

NarrowDiffBox(4)

In [98]:
d5 == NarrowDiffBox(5)

NarrowDiffBox.__eq__(NarrowDiffBox(5), NarrowDiffBox(5))


True

In [99]:
d5 == NarrowDiffBox(4)

NarrowDiffBox.__eq__(NarrowDiffBox(5), NarrowDiffBox(4))


False

In [100]:
d20 = NarrowDiffBox(20)

In [101]:
d10 = NarrowDiffBox(10)

In [102]:
d20

NarrowDiffBox(20)

In [103]:
wd20 = weakref.proxy(d20)
# This or any comment works around a very strange IPython bug.

In [104]:
wd20

<weakproxy at 0x0000025B31F7E9D0 to NarrowDiffBox at 0x0000025B3267B950>

In [105]:
wd10 = weakref.proxy(d10)
# This or any comment works around a very strange IPython bug.

In [106]:
wd20 == wd10

NarrowDiffBox.__eq__(NarrowDiffBox(20), NarrowDiffBox(10))


False

In [107]:
wd20 - wd10

NarrowDiffBox.__sub__(NarrowDiffBox(20), NarrowDiffBox(10))


NarrowDiffBox(10)

In [108]:
result = wd20 - wd10

NarrowDiffBox.__sub__(NarrowDiffBox(20), NarrowDiffBox(10))


In [109]:
result - wd10

Typecheck denied in NarrowDiffBox.__sub__(NarrowDiffBox(10), <weakproxy at 0x0000025B326D14E0 to NarrowDiffBox at 0x0000025B325D6210>)
NarrowDiffBox.__sub__(NarrowDiffBox(10), NarrowDiffBox(10))


NarrowDiffBox(0)

In [110]:
d20

NarrowDiffBox(20)

In [111]:
wd10

<weakproxy at 0x0000025B326D14E0 to NarrowDiffBox at 0x0000025B325D6210>

In [112]:
d20 - wd10

Typecheck denied in NarrowDiffBox.__sub__(NarrowDiffBox(20), <weakproxy at 0x0000025B326D14E0 to NarrowDiffBox at 0x0000025B325D6210>)
NarrowDiffBox.__sub__(NarrowDiffBox(20), NarrowDiffBox(10))


NarrowDiffBox(10)

In [113]:
NarrowDiffBox(50) - weakref.proxy(d10)

Typecheck denied in NarrowDiffBox.__sub__(NarrowDiffBox(50), <weakproxy at 0x0000025B326D14E0 to NarrowDiffBox at 0x0000025B325D6210>)
NarrowDiffBox.__sub__(NarrowDiffBox(50), NarrowDiffBox(10))


NarrowDiffBox(40)

In [114]:
weakref.proxy(d10) - NarrowDiffBox(5)

NarrowDiffBox.__sub__(NarrowDiffBox(10), NarrowDiffBox(5))


NarrowDiffBox(5)

In [115]:
wd10.__rsub__

AttributeError: 'NarrowDiffBox' object has no attribute '__rsub__'

In [ ]:
d10 - weakref.proxy(d20)

In [ ]:
d10.__sub__(weakref.proxy(d20))

In [ ]:
weakref.proxy(d20).__rsub__(d10)

In [118]:
type(wd20).__rsub__

<slot wrapper '__rsub__' of 'weakref.ProxyType' objects>

In [121]:
type(wd20).__rsub__(wd20, wd10)

NarrowDiffBox.__sub__(NarrowDiffBox(10), NarrowDiffBox(20))


NarrowDiffBox(-10)

Sidebar: Review of metaclass confusion

```python
x - y  # What does this do?
x.__sub__(y)  # We think of it like this, but this actually never happens.
type(x).__sub__(x, y)  # Usually this is tried first.
type(y).__rsub__(y, x)  # Then this is tried if __sub__ returned NotImplemented.
```

In [119]:
x, y = 2, 1

In [120]:
x.__sub__(y)

1